In [1]:
%load_ext autoreload
%autoreload 2
import pickle
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from utils.midi import *
from utils.data import *

In [ ]:
path = './data/midi/'
dfs = get_dfs_from_midi(path, min_notes=50, min_gap=0.)

In [ ]:
with open('./data/dfs_min_notes_50_min_gap_0.pkl', 'wb') as f:
    pickle.dump(dfs, f)

# Number of notes

In [ ]:
lens = [len(df) for df in dfs]
print('Number of songs:', len(dfs))
print('Max length:', max(lens))
print('Min length:', min(lens))
print('Mean length:', np.mean(lens))
sorted_lens = sorted(lens)
plt.bar(range(len(sorted_lens)), sorted_lens);

# Duration

In [ ]:
# Duration of melodies
durations = []
for df in dfs:
    durations.append(df['End'].max())
plt.hist(durations, bins=100);
min(durations), max(durations)

# Pitch range

In [ ]:
pitch_range(dfs)

In [ ]:
dfs2 = trim_by_range(dfs, min_range=5, max_range=26)

In [ ]:
pitch_range(dfs2)

In [ ]:
dfs3 = move_octaves(dfs2, min_pitch=48, center_range=[61,72])

In [ ]:
pitch_range(dfs3)

In [ ]:
with open('./data/dataframes/min_note_50_min_gap_0/dfs_pitch_range_5_26_min_pitch_48_center_61_72.pkl', 'wb') as f:
    pickle.dump(dfs3, f)

# Octave translations

In [ ]:
sample_idx = 400
dfs3[sample_idx]

In [ ]:
Fs = 44100
midi_data = df_to_midi(dfs[sample_idx])
audio_data = midi_data.synthesize(fs=Fs)
ipd.Audio(audio_data, rate=Fs)

In [ ]:
df_octave_down = dfs[sample_idx].copy()
df_octave_down['Pitch'] = df_octave_down['Pitch'] - 12
midi_data = df_to_midi(df_octave_down)
audio_data = midi_data.synthesize(fs=Fs)
ipd.Audio(audio_data, rate=Fs)

# Gaps

In [ ]:
# measure gap between start of next note and end of previous note
neg_gaps = []
pos_gaps = []
for df in dfs:
    gap = df['Start'].iloc[1:].values - df['End'].iloc[:-1].values
    if gap.min() < 0:
        neg_gaps.append(df)
    if gap.max() > 0:
        pos_gaps.append(df)
print(len(neg_gaps), len(pos_gaps))

# Duration + Pitch format

In [ ]:
dfs4 = note_duration_transform(dfs3)

In [ ]:
dfs5, max_pitch = pitch_translation(dfs4)

In [ ]:
pitch_range(dfs5)

In [ ]:
with open('./data/dataframes/min_note_50_min_gap_0/dfs_note_dur_offset_47.pkl', 'wb') as f:
    pickle.dump(dfs5, f)

In [2]:
with open('./data/dataframes/min_note_50_min_gap_0/dfs_note_dur_offset_47.pkl', 'rb') as f:
    dfs5 = pickle.load(f)

In [4]:
dataset = NoteDurationDataset(dfs5, sample_len=50, scale=1., stride=10)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=0, drop_last=True)

In [11]:
len(dataloader)

345